In [ ]:
import math
import time
from typing import List, Dict, Any, Optional
import argparse
import random
import os
import json

import torch
import torch.nn as nn
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers.models.llava.image_processing_llava import LlavaImageProcessor

from dataset import VQAv2Eval
import transformers

/fs/nexus-scratch/vla/micromamba/envs/MMQ_LLAVA/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(transformers.__version__)

4.51.3


In [3]:
DEBUG = False

torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False

In [3]:
#

# ====================================================
# Quantization Classes and Functions
# ====================================================

def quantize(x, scale, zero, maxq):
    if maxq < 0:
        return (x > scale / 2).float() * scale + (x < zero / 2).float() * zero
    q = torch.clamp(torch.round(x / scale) + zero, 0, maxq)
    return scale * (q - zero)


class Quantizer(nn.Module):
    def __init__(self, shape=1):
        super(Quantizer, self).__init__()
        self.register_buffer("maxq", torch.tensor(0))
        self.register_buffer("scale", torch.zeros(shape))
        self.register_buffer("zero", torch.zeros(shape))

    def configure(
        self,
        bits,
        perchannel=False,
        sym=True,
        mse=False,
        norm=2.4,
        grid=100,
        maxshrink=0.8,
        trits=False,
    ):
        device = self.maxq.device
        self.maxq = torch.tensor(2**bits - 1, device=device)
        self.perchannel = perchannel
        self.sym = sym
        self.mse = mse
        self.norm = norm
        self.grid = grid
        self.maxshrink = maxshrink
        if trits:
            self.maxq = torch.tensor(-1, device=device)

    def find_params(self, x, weight=False):
        dev = x.device
        self.maxq = self.maxq.to(dev)

        shape = x.shape
        if self.perchannel:
            if weight:
                x = x.flatten(1)
            else:
                if len(shape) == 4:
                    x = x.permute([1, 0, 2, 3])
                    x = x.flatten(1)
                if len(shape) == 3:
                    x = x.reshape((-1, shape[-1])).t()
                if len(shape) == 2:
                    x = x.t()
        else:
            x = x.flatten().unsqueeze(0)

        tmp = torch.zeros(x.shape[0], device=dev)
        xmin = torch.minimum(x.min(1)[0], tmp)
        xmax = torch.maximum(x.max(1)[0], tmp)

        if self.sym:
            xmax = torch.maximum(torch.abs(xmin), xmax)
            tmp = xmin < 0
            if torch.any(tmp):
                xmin[tmp] = -xmax[tmp]
        tmp = (xmin == 0) & (xmax == 0)
        xmin[tmp] = -1
        xmax[tmp] = +1

        if self.maxq < 0:
            self.scale = xmax
            self.zero = xmin
        else:
            self.scale = (xmax - xmin) / self.maxq
            if self.sym:
                self.zero = torch.full_like(self.scale, (self.maxq + 1) / 2)
            else:
                self.zero = torch.round(-xmin / self.scale)

        if self.mse:
            best = torch.full([x.shape[0]], float("inf"), device=dev)
            for i in range(int(self.maxshrink * self.grid)):
                p = 1 - i / self.grid
                xmin1 = p * xmin
                xmax1 = p * xmax
                scale1 = (xmax1 - xmin1) / self.maxq
                zero1 = torch.round(-xmin1 / scale1) if not self.sym else self.zero
                q = quantize(x, scale1.unsqueeze(1), zero1.unsqueeze(1), self.maxq)
                q -= x
                q.abs_()
                q.pow_(self.norm)
                err = torch.sum(q, 1)
                tmp = err < best
                if torch.any(tmp):
                    best[tmp] = err[tmp]
                    self.scale[tmp] = scale1[tmp]
                    self.zero[tmp] = zero1[tmp]
        if not self.perchannel:
            if weight:
                tmp = shape[0]
            else:
                tmp = shape[1] if len(shape) != 3 else shape[2]
            self.scale = self.scale.repeat(tmp)
            self.zero = self.zero.repeat(tmp)

        if weight:
            shape = [-1] + [1] * (len(shape) - 1)
            self.scale = self.scale.reshape(shape)
            self.zero = self.zero.reshape(shape)
            return
        if len(shape) == 4:
            self.scale = self.scale.reshape((1, -1, 1, 1))
            self.zero = self.zero.reshape((1, -1, 1, 1))
        if len(shape) == 3:
            self.scale = self.scale.reshape((1, 1, -1))
            self.zero = self.zero.reshape((1, 1, -1))
        if len(shape) == 2:
            self.scale = self.scale.unsqueeze(0)
            self.zero = self.zero.unsqueeze(0)

        # Ensure buffers are on the same device as input x
        self.scale = self.scale.to(dev)
        self.zero = self.zero.to(dev)

    def quantize(self, x):
        if self.ready():
            # Ensure buffers are on the same device as x
            self.scale = self.scale.to(x.device)
            self.zero = self.zero.to(x.device)
            self.maxq = self.maxq.to(x.device)
            return quantize(x, self.scale, self.zero, self.maxq)
        return x

    def enabled(self):
        return self.maxq > 0

    def ready(self):
        return torch.all(self.scale != 0)


class GPTQ:
    def __init__(self, layer):
        self.layer = layer
        self.dev = self.layer.weight.device
        W = layer.weight.data.clone()
        if isinstance(self.layer, nn.Conv2d):
            W = W.flatten(1)
        if isinstance(self.layer, transformers.Conv1D):
            W = W.t()
        self.rows = W.shape[0]
        self.columns = W.shape[1]
        self.H = torch.zeros((self.columns, self.columns), device=self.dev)
        self.nsamples = 0
        self.quantizer = Quantizer()
        self.quantizer.to(self.dev)

    def add_batch(self, inp, out):
        if DEBUG:
            self.inp1 = inp
            self.out1 = out
        if len(inp.shape) == 2:
            inp = inp.unsqueeze(0)
        tmp = inp.shape[0]
        if isinstance(self.layer, nn.Linear) or isinstance(
            self.layer, transformers.Conv1D
        ):
            if len(inp.shape) == 3:
                inp = inp.reshape((-1, inp.shape[-1]))
            inp = inp.t()
        if isinstance(self.layer, nn.Conv2d):
            unfold = nn.Unfold(
                self.layer.kernel_size,
                dilation=self.layer.dilation,
                padding=self.layer.padding,
                stride=self.layer.stride,
            )
            inp = unfold(inp)
            inp = inp.permute([1, 0, 2])
            inp = inp.flatten(1)

        self.H *= self.nsamples / (self.nsamples + tmp)
        self.nsamples += tmp
        inp = math.sqrt(2 / self.nsamples) * inp.float()
        self.H += inp.matmul(inp.t())

    def fasterquant(
        self,
        blocksize=128,
        percdamp=0.01,
        groupsize=-1,
        actorder=False,
        static_groups=False,
    ):
        W = self.layer.weight.data.clone()
        if isinstance(self.layer, nn.Conv2d):
            W = W.flatten(1)
        if isinstance(self.layer, transformers.Conv1D):
            W = W.t()
        W = W.float()

        tick = time.time()

        if not self.quantizer.ready():
            self.quantizer.find_params(W, weight=True)

        H = self.H
        del self.H
        dead = torch.diag(H) == 0
        H[dead, dead] = 1
        W[:, dead] = 0

        if static_groups:
            import copy

            groups = []
            for i in range(0, self.columns, groupsize):
                quantizer = copy.deepcopy(self.quantizer)
                quantizer.find_params(W[:, i : (i + groupsize)], weight=True)
                groups.append(quantizer)

        if actorder:
            perm = torch.argsort(torch.diag(H), descending=True)
            W = W[:, perm]
            H = H[perm][:, perm]
            invperm = torch.argsort(perm)

        Losses = torch.zeros_like(W)
        Q = torch.zeros_like(W)

        damp = percdamp * torch.mean(torch.diag(H))
        diag = torch.arange(self.columns, device=self.dev)
        H[diag, diag] += damp
        H = torch.linalg.cholesky(H)
        H = torch.cholesky_inverse(H)
        H = torch.linalg.cholesky(H, upper=True)
        Hinv = H

        for i1 in range(0, self.columns, blocksize):
            i2 = min(i1 + blocksize, self.columns)
            count = i2 - i1

            W1 = W[:, i1:i2].clone()
            Q1 = torch.zeros_like(W1)
            Err1 = torch.zeros_like(W1)
            Losses1 = torch.zeros_like(W1)
            Hinv1 = Hinv[i1:i2, i1:i2]

            for i in range(count):
                w = W1[:, i]
                d = Hinv1[i, i]

                if groupsize != -1:
                    if not static_groups:
                        if (i1 + i) % groupsize == 0:
                            self.quantizer.find_params(
                                W[:, (i1 + i) : (i1 + i + groupsize)], weight=True
                            )
                    else:
                        idx = i1 + i
                        if actorder:
                            idx = perm[idx]
                        self.quantizer = groups[idx // groupsize]

                q = quantize(
                    w.unsqueeze(1),
                    self.quantizer.scale,
                    self.quantizer.zero,
                    self.quantizer.maxq,
                ).flatten()
                Q1[:, i] = q
                Losses1[:, i] = (w - q) ** 2 / d**2

                err1 = (w - q) / d
                W1[:, i:] -= err1.unsqueeze(1).matmul(Hinv1[i, i:].unsqueeze(0))
                Err1[:, i] = err1

            Q[:, i1:i2] = Q1
            Losses[:, i1:i2] = Losses1 / 2

            W[:, i2:] -= Err1.matmul(Hinv[i1:i2, i2:])

            if DEBUG:
                self.layer.weight.data[:, :i2] = Q[:, :i2]
                self.layer.weight.data[:, i2:] = W[:, i2:]
                print(torch.sum((self.layer(self.inp1) - self.out1) ** 2))
                print(torch.sum(Losses))

        torch.cuda.synchronize()
        print("Time for quantization: %.2f seconds" % (time.time() - tick))
        print("Total quantization error:", torch.sum(Losses).item())

        if actorder:
            Q = Q[:, invperm]

        if isinstance(self.layer, transformers.Conv1D):
            Q = Q.t()
        self.layer.weight.data = Q.reshape(self.layer.weight.shape).to(
            self.layer.weight.data.dtype
        )
        if DEBUG:
            print(torch.sum((self.layer(self.inp1) - self.out1) ** 2))

    def free(self):
        if DEBUG:
            self.inp1 = None
            self.out1 = None
        self.H = None
        self.Losses = None
        self.Trace = None
        torch.cuda.empty_cache()


def find_linear_layers_in_model(model):
    layers = {}

    def recurse(module, prefix=""):
        if isinstance(module, nn.Linear):
            layers[prefix.rstrip(".")] = module
        for name, child in module.named_children():
            recurse(child, prefix + name + ".")

    recurse(model)
    return layers


In [4]:
class LlavaQuantizer:
    def __init__(self, model, processor, device, chunk_size=32, task = 'vqav2'):
        self.model = model
        self.processor = processor
        self.device = device
        self.chunk_size = chunk_size
        self.task = task

        # Component-specific configuration parameters
        self.config = {
            "vision": {
                "bits": 4,
                "percent_dampening": 0.01,
                "group_size": -1,
                "use_symmetric": True,
                "use_act_order": False,
                "use_static_groups": False,
            },
            "language": {
                "bits": 4,
                "percent_dampening": 0.01,
                "group_size": -1,
                "use_symmetric": True,
                "use_act_order": False,
                "use_static_groups": False,
            },
        }


    def _prepare_quantizers(self, layers, component_type):
        """Initialize GPTQ quantizers for given layers with component-specific settings"""
        config = self.config[component_type]
        quantizers = {}
        for name, layer in layers.items():
            quantizers[name] = GPTQ(layer)
            quantizers[name].quantizer.configure(
                bits=config["bits"],
                perchannel=True,
                sym=config["use_symmetric"],
                mse=False,
            )
        return quantizers
    
    def _process_chunk(
        self, layers, start_idx, end_idx, forward_func, desc, component_type
    ):
        """Process a chunk of layers with component-specific quantization settings"""
        current_layers = dict(list(layers.items())[start_idx:end_idx])
        print(
            f"\nProcessing {desc} layers {start_idx} to {end_idx-1} with {self.config[component_type]['bits']}-bit precision"
        )

        # Initialize quantizers for current chunk
        quantizers = self._prepare_quantizers(current_layers, component_type)
        hooks = []

        def get_hook(name):
            def hook(module, inp, out):
                if name in quantizers:
                    quantizers[name].add_batch(inp[0].detach(), out.detach())

            return hook

        for name, layer in current_layers.items():
            hooks.append(layer.register_forward_hook(get_hook(name)))

        forward_func()

        for hook in hooks:
            hook.remove()

        config = self.config[component_type]
        for name, layer in current_layers.items():
            print(f"Quantizing layer {name}...")
            quantizer = quantizers[name]
            quantizer.fasterquant(
                blocksize=32,
                percdamp=config["percent_dampening"],
                groupsize=config["group_size"],
                actorder=config["use_act_order"],
                static_groups=config["use_static_groups"],
            )

            layer.weight.data = quantizer.quantizer.quantize(layer.weight.data).to(
                layer.weight.data.dtype
            )
            quantizer.free()

        torch.cuda.empty_cache()


    def quantize_vision_model(self, calibration_set):
        """Quantize vision model with 8-bit precision"""
        print(
            f"Quantizing Vision Model with {self.config['vision']['bits']}-bit precision..."
        )

        # some extra components need to be on device for vision model forward pass
        # self.model.vision_tower.to(self.device)
        self.model.to(self.device)
        self.model.language_model.to('cpu')

        layers = find_linear_layers_in_model(self.model.vision_tower.vision_model)
        total_layers = len(layers)

        print(f'total_layers: {total_layers}')
        print(layers)

        def forward_pass():
            
            vision_feature_layer = self.model.config.vision_feature_layer
            vision_feature_select_strategy = self.model.config.vision_feature_select_strategy
            image_sizes = None
            
            # TODO: adjust for GQA if needed
            if self.task == 'vqav2':
                
                for img, prompt in tqdm(calibration_set, desc='Processing vision model batch'):

                    inputs = self.processor(images = [img],
                                            text= [prompt],
                                            return_tensors='pt',
                                            padding=True).to(self.device)
                    
                    # runs forward pass through vision_tower
                    self.model.get_image_features(
                        pixel_values = inputs['pixel_values'],
                        vision_feature_layer=vision_feature_layer,
                        vision_feature_select_strategy=vision_feature_select_strategy,
                        image_sizes=image_sizes
                    )


        for start_idx in range(0, total_layers, self.chunk_size):
            end_idx = min(start_idx + self.chunk_size, total_layers)
            self._process_chunk(
                layers, start_idx, end_idx, forward_pass, "vision model", "vision"
            )

        self.model.vision_tower.vision_model.cpu()
        print("Vision Model quantization complete.\n")


    def quantize_language_model(self, calibration_set):
        """Quantize language model with 4-bit precision"""
        print(
            f"Quantizing Language Model with {self.config['language']['bits']}-bit precision..."
        )
        self.model.to(self.device)

        layers = find_linear_layers_in_model(self.model.language_model.model)
        # layers["language_projection"] = self.model.language_projection
        total_layers = len(layers)

        def forward_pass():
            # TODO: adjust for GQA if needed
            if self.task == 'vqav2':
                
                for img, prompt in tqdm(calibration_set, desc='Processing language model batch'):

                    inputs = self.processor(images = [img],
                                            text= [prompt],
                                            return_tensors='pt',
                                            padding=True).to(self.device)
                    
                    self.model.generate(**inputs)
   

        for start_idx in range(0, total_layers, self.chunk_size):
            end_idx = min(start_idx + self.chunk_size, total_layers)
            self._process_chunk(
                layers, start_idx, end_idx, forward_pass, "language model", "language"
            )

        self.model.cpu()
        print("Language Model quantization complete.\n")

    def quantize(self, calibration_set):
        """Quantize all LLAVA components"""
        print("Starting LLAVA model quantization...")
        self.quantize_vision_model(calibration_set)
        self.quantize_language_model(calibration_set)
        print("LLAVA model quantization complete.")


    # TODO:
    def prepare_for_inference(self):
        self.model.to(self.device)


In [10]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
print(device)

# Load the model
model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf", torch_dtype=torch.float16)
model.to('cpu')
# processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf", pad_token = '<pad>')
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf", pad_token = '<pad>', use_fast = False)


image_processor = LlavaImageProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf",
                                                      do_pad=True)

cuda


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 18.58it/s]


In [13]:
processor.image_processor = image_processor

In [14]:
processor

LlavaProcessor:
- image_processor: LlavaImageProcessor {
  "crop_size": {
    "height": 336,
    "width": 336
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_pad": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "LlavaImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "processor_class": "LlavaProcessor",
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 336
  }
}

- tokenizer: LlamaTokenizer(name_or_path='llava-hf/llava-1.5-7b-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'image_token': '<image>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=Fals

In [15]:
# # VQAv2 dataset paths
# ann_root = '/fs/cfar-projects/low-bit-vision/datasets/vqav2/annotations'
# q_root = '/fs/cfar-projects/low-bit-vision/datasets/vqav2/questions'
# image_root = '/fs/cfar-projects/low-bit-vision/datasets/vqav2/val2014'

# # short answer prompting according to: https://github.com/haotian-liu/LLaVA/blob/main/docs/Evaluation.md
llava_prompt = 'USER: <image>\n{}\nAnswer the question using a single word or phrase. ASSISTANT:'

# dataset = VQAv2Eval(image_root=image_root,
#                     ann_root=ann_root,
#                     q_root=q_root,
#                     prompt=llava_prompt)


from dataset import GQAEval

image_root = '/fs/cfar-projects/low-bit-vision/datasets/gqa/images'
q_root = '/fs/cfar-projects/low-bit-vision/datasets/gqa/questions'

dataset = GQAEval(
        image_root,
        q_root,
        prompt=llava_prompt
)

In [16]:
dataset[110]

{'question_id': '20300425',
 'image': <PIL.Image.Image image mode=RGB size=500x378>,
 'text_input': 'USER: <image>\nWhich kind of vehicle is waiting for the traffic light?\nAnswer the question using a single word or phrase. ASSISTANT:',
 'gt_answer': 'cars'}

In [ ]:
dataloader = DataLoader(dataset,
                        batch_size=16,
                        num_workers=1,
                        pin_memory=False,
                        shuffle=False,
                        collate_fn = dataset.collater)

In [6]:
from inference_pipeline import InferencePipeline
inferencer = InferencePipeline(model, device, processor)
processor_kwargs = dict(padding=True)
model.to(device)

LlavaForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): Q

In [18]:
# results = inferencer.run_inference(
#     dataloader,
#     task = 'vqav2',
#     processor_kwargs = processor_kwargs,
#     generate_kwargs = None
# )

# results = inferencer.run_inference(
#     dataloader,
#     task = 'gqa',
#     processor_kwargs = processor_kwargs,
#     generate_kwargs = None
# )

In [13]:
total_indices = range(len(dataset))
total_indices

range(0, 214354)

In [15]:
CALIBRATION_SIZE = 128
calibration_indices = random.sample(total_indices, CALIBRATION_SIZE)

calibration_set = [(dataset[i]['image'], dataset[i]['text_input']) for i in calibration_indices]
len(calibration_set)

128

In [9]:
quantizer = LlavaQuantizer(model, processor, device)

quantizer.config = {
    "vision": {
        "bits": 2,
        "percent_dampening": 0.01,
        "group_size": -1,
        "use_symmetric": True,
        "use_act_order": False,
        "use_static_groups": False,
    },
    "language": {
        "bits": 6,
        "percent_dampening": 0.01,
        "group_size": -1,
        "use_symmetric": True,
        "use_act_order": False,
        "use_static_groups": False,
    },
}

# quantizer.quantize_vision_model(calibration_set)
# quantizer.quantize_language_model(calibration_set)

start_time = time.time()
quantizer.quantize(calibration_set)

print(f'Elapsed time: {time.time() - start_time}')

Starting LLAVA model quantization...
Quantizing Vision Model with 2-bit precision...
total_layers: 144
{'encoder.layers.0.self_attn.k_proj': Linear(in_features=1024, out_features=1024, bias=True), 'encoder.layers.0.self_attn.v_proj': Linear(in_features=1024, out_features=1024, bias=True), 'encoder.layers.0.self_attn.q_proj': Linear(in_features=1024, out_features=1024, bias=True), 'encoder.layers.0.self_attn.out_proj': Linear(in_features=1024, out_features=1024, bias=True), 'encoder.layers.0.mlp.fc1': Linear(in_features=1024, out_features=4096, bias=True), 'encoder.layers.0.mlp.fc2': Linear(in_features=4096, out_features=1024, bias=True), 'encoder.layers.1.self_attn.k_proj': Linear(in_features=1024, out_features=1024, bias=True), 'encoder.layers.1.self_attn.v_proj': Linear(in_features=1024, out_features=1024, bias=True), 'encoder.layers.1.self_attn.q_proj': Linear(in_features=1024, out_features=1024, bias=True), 'encoder.layers.1.self_attn.out_proj': Linear(in_features=1024, out_feature

Processing vision model batch: 100%|██████████| 128/128 [00:08<00:00, 15.59it/s]


Quantizing layer encoder.layers.0.self_attn.k_proj...
Time for quantization: 1.72 seconds
Total quantization error: 7536.826171875
Quantizing layer encoder.layers.0.self_attn.v_proj...
Time for quantization: 0.28 seconds
Total quantization error: 5068.3720703125
Quantizing layer encoder.layers.0.self_attn.q_proj...
Time for quantization: 0.27 seconds
Total quantization error: 8809.1796875
Quantizing layer encoder.layers.0.self_attn.out_proj...
Time for quantization: 0.28 seconds
Total quantization error: 36.56419372558594
Quantizing layer encoder.layers.0.mlp.fc1...
Time for quantization: 0.30 seconds
Total quantization error: 138271.296875
Quantizing layer encoder.layers.0.mlp.fc2...
Time for quantization: 1.14 seconds
Total quantization error: 801.8564453125
Quantizing layer encoder.layers.1.self_attn.k_proj...
Time for quantization: 0.27 seconds
Total quantization error: 7883.9990234375
Quantizing layer encoder.layers.1.self_attn.v_proj...
Time for quantization: 0.28 seconds
Total q

Processing vision model batch: 100%|██████████| 128/128 [00:03<00:00, 42.29it/s]


Quantizing layer encoder.layers.5.self_attn.q_proj...
Time for quantization: 0.28 seconds
Total quantization error: 30146.412109375
Quantizing layer encoder.layers.5.self_attn.out_proj...
Time for quantization: 0.27 seconds
Total quantization error: 209.6897430419922
Quantizing layer encoder.layers.5.mlp.fc1...
Time for quantization: 0.27 seconds
Total quantization error: 137523.125
Quantizing layer encoder.layers.5.mlp.fc2...
Time for quantization: 1.10 seconds
Total quantization error: 1430.869873046875
Quantizing layer encoder.layers.6.self_attn.k_proj...
Time for quantization: 0.27 seconds
Total quantization error: 79953.078125
Quantizing layer encoder.layers.6.self_attn.v_proj...
Time for quantization: 0.28 seconds
Total quantization error: 22287.55859375
Quantizing layer encoder.layers.6.self_attn.q_proj...
Time for quantization: 0.31 seconds
Total quantization error: 58968.96875
Quantizing layer encoder.layers.6.self_attn.out_proj...
Time for quantization: 0.30 seconds
Total qua

...

Quantizing layer layers.18.self_attn.v_proj...
Time for quantization: 1.14 seconds
Total quantization error: 67.752197265625
Quantizing layer layers.18.self_attn.o_proj...
Time for quantization: 1.13 seconds
Total quantization error: 1.2422922849655151
Quantizing layer layers.18.mlp.gate_proj...
Time for quantization: 1.23 seconds
Total quantization error: 127.36318969726562
Quantizing layer layers.18.mlp.up_proj...
Time for quantization: 1.17 seconds
Total quantization error: 111.65200805664062
Quantizing layer layers.18.mlp.down_proj...
Time for quantization: 3.40 seconds
Total quantization error: 16.19788360595703
Quantizing layer layers.19.self_attn.q_proj...
Time for quantization: 1.79 seconds
Total quantization error: 192.74815368652344
Quantizing layer layers.19.self_attn.k_proj...
Time for quantization: 1.12 seconds
Total quantization error: 153.8065185546875
Quantizing layer layers.19.self_attn.v_proj...
Time for quantization: 1.13 seconds
Total quantization error: 68.77992248

Processing language model batch: 100%|██████████| 128/128 [00:44<00:00,  2.89it/s]


Quantizing layer layers.22.mlp.down_proj...
Time for quantization: 3.46 seconds
Total quantization error: 17.65645980834961
Quantizing layer layers.23.self_attn.q_proj...
Time for quantization: 1.13 seconds
Total quantization error: 260.3820495605469
Quantizing layer layers.23.self_attn.k_proj...
Time for quantization: 1.12 seconds
Total quantization error: 196.86280822753906
Quantizing layer layers.23.self_attn.v_proj...
Time for quantization: 1.12 seconds
Total quantization error: 104.8172836303711
Quantizing layer layers.23.self_attn.o_proj...
Time for quantization: 1.13 seconds
Total quantization error: 1.3783042430877686
Quantizing layer layers.23.mlp.gate_proj...
Time for quantization: 1.17 seconds
Total quantization error: 180.47747802734375
Quantizing layer layers.23.mlp.up_proj...
Time for quantization: 1.17 seconds
Total quantization error: 154.5054473876953
Quantizing layer layers.23.mlp.down_proj...
Time for quantization: 3.38 seconds
Total quantization error: 20.9155349731

Processing language model batch: 100%|██████████| 128/128 [00:44<00:00,  2.88it/s]


Quantizing layer layers.27.self_attn.o_proj...
Time for quantization: 1.13 seconds
Total quantization error: 1.2542890310287476
Quantizing layer layers.27.mlp.gate_proj...
Time for quantization: 1.17 seconds
Total quantization error: 191.4755096435547
Quantizing layer layers.27.mlp.up_proj...
Time for quantization: 1.23 seconds
Total quantization error: 166.24130249023438
Quantizing layer layers.27.mlp.down_proj...
Time for quantization: 3.39 seconds
Total quantization error: 19.493118286132812
Quantizing layer layers.28.self_attn.q_proj...
Time for quantization: 1.14 seconds
Total quantization error: 332.40203857421875
Quantizing layer layers.28.self_attn.k_proj...
Time for quantization: 1.14 seconds
Total quantization error: 185.15736389160156
Quantizing layer layers.28.self_attn.v_proj...
Time for quantization: 1.13 seconds
Total quantization error: 117.61782836914062
Quantizing layer layers.28.self_attn.o_proj...
Time for quantization: 1.12 seconds
Total quantization error: 1.71962

In [16]:
img = dataset[42]['image']
# prompt = 'USER: <image>\n' + dataset.qa_pairs[42]['question'] + '\nAnswer the question using a single word or phrase. ASSISTANT:'

prompt = dataset[42]['text_input']
print(prompt)

model.to('cuda')

# set this according to huggingface usage tips: https://huggingface.co/docs/transformers/en/model_doc/llava
processor.tokenizer.padding_side = "left"
samples = processor(images = [img],
                     text=[prompt],
                     return_tensors='pt',
                     padding=True).to(model.device)

# Generate
# generate_ids = model.generate(**inputs, max_new_tokens=30)
generate_ids = model.generate(**samples, max_new_tokens=30)
output = processor.batch_decode(generate_ids, skip_special_tokens=True)
output

USER: <image>
How is the clothing item that is pink called?
Answer the question using a single word or phrase. ASSISTANT:


['USER: \nHow is the clothing item that is pink called?\nAnswer the question using a single word or phrase. ASSISTANT: Tank top']

In [12]:
output[0].split('ASSISTANT: ')[-1]

'Tank top'

In [8]:
dataset[42]

{'question_id': '201974971',
 'image': <PIL.Image.Image image mode=RGB size=640x427>,
 'text_input': 'USER: <image>\nHow is the clothing item that is pink called?\nAnswer the question using a single word or phrase. ASSISTANT:',
 'gt_answer': 'tank top'}